## Covid19 Tweets EDA and Text Sentiment Analysis

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
# import nltk

%matplotlib inline
plt.style.use('ggplot')

In [3]:
#To set up JupyterLab
#1. Make sure docker is running. If another container is running, use 'docker rm [name]' to remove it
#2. Run docker container:
#   'sudo docker run -d --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token='' '
#   'docker exec -it sparkbook bash'
#3. Open browser and go to 'localhost:8881'

In [5]:
#Setting up pyspark session
import pyspark as ps

spark = (ps.sql.SparkSession
            .builder
            .master('local[4]')
            .appName('Covid19_tweets')
            .getOrCreate())

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=Covid19_tweets>

In [7]:
covid_data = pd.read_csv('covid19_tweets.csv')
covid_data.head()

user_name         user_location  \
0           ᏉᎥ☻լꂅϮ            astroworld   
1    Tom Basile 🇺🇸          New York, NY   
2  Time4fisticuffs      Pewee Valley, KY   
3      ethel mertz  Stuck in the Middle    
4         DIPR-J&K     Jammu and Kashmir   

                                    user_description         user_created  \
0  wednesday addams as a disney princess keepin i...  2017-05-26 05:46:42   
1  Husband, Father, Columnist & Commentator. Auth...  2009-04-16 20:06:23   
2  #Christian #Catholic #Conservative #Reagan #Re...  2009-02-28 18:57:41   
3  #Browns #Indians #ClevelandProud #[]_[] #Cavs ...  2019-03-07 01:45:06   
4  🖊️Official Twitter handle of Department of Inf...  2017-02-12 06:45:15   

   user_followers  user_friends  user_favourites  user_verified  \
0             624           950            18775          False   
1            2253          1677               24           True   
2            9275          9525             7254          False   
3             197           987             1488          False   
4          101009           168              101          False   

                  date                                               text  \
0  2020-07-25 12:27:21  If I smelled the scent of hand sanitizers toda...   
1  2020-07-25 12:27:17  Hey @Yankees @YankeesPR and @MLB - wouldn't it...   
2  2020-07-25 12:27:14  @diane3443 @wdunlap @realDonaldTrump Trump nev...   
3  2020-07-25 12:27:10  @brookbanktv The one gift #COVID19 has give me...   
4  2020-07-25 12:27:08  25 July : Media Bulletin on Novel #CoronaVirus...   

                            hashtags               source  is_retweet  
0                                NaN   Twitter for iPhone       False  
1                                NaN  Twitter for Android       False  
2                        ['COVID19']  Twitter for Android       False  
3                        ['COVID19']   Twitter for iPhone       False  
4  ['CoronaVirusUpdates', 'COVID19']  Twitter for Android       False

In [8]:
covid_data.shape

(140101, 13)

In [9]:
#Removing duplicates
covid_data.drop_duplicates()

user_name         user_location  \
0                ᏉᎥ☻լꂅϮ            astroworld   
1         Tom Basile 🇺🇸          New York, NY   
2       Time4fisticuffs      Pewee Valley, KY   
3           ethel mertz  Stuck in the Middle    
4              DIPR-J&K     Jammu and Kashmir   
...                 ...                   ...   
140096     Aaron Derfel              Montreal   
140097    Pierre Alozie                 World   
140098    Pierre Alozie                 World   
140099    Pierre Alozie                 World   
140100    Pierre Alozie                 World   

                                         user_description  \
0       wednesday addams as a disney princess keepin i...   
1       Husband, Father, Columnist & Commentator. Auth...   
2       #Christian #Catholic #Conservative #Reagan #Re...   
3       #Browns #Indians #ClevelandProud #[]_[] #Cavs ...   
4       🖊️Official Twitter handle of Department of Inf...   
...                                                   ...   
140096  I'm the health-care journalist at the Montreal...   
140097                     Photojournalist @ F8, FFR, NUJ   
140098                     Photojournalist @ F8, FFR, NUJ   
140099                     Photojournalist @ F8, FFR, NUJ   
140100                     Photojournalist @ F8, FFR, NUJ   

               user_created  user_followers  user_friends  user_favourites  \
0       2017-05-26 05:46:42             624           950            18775   
1       2009-04-16 20:06:23            2253          1677               24   
2       2009-02-28 18:57:41            9275          9525             7254   
3       2019-03-07 01:45:06             197           987             1488   
4       2017-02-12 06:45:15          101009           168              101   
...                     ...             ...           ...              ...   
140096  2010-07-01 01:19:14           18063           197             6083   
140097  2009-03-20 12:27:35             300          1002              703   
140098  2009-03-20 12:27:35             300          1002              703   
140099  2009-03-20 12:27:35             300          1002              703   
140100  2009-03-20 12:27:35             300          1002              703   

        user_verified                 date  \
0               False  2020-07-25 12:27:21   
1                True  2020-07-25 12:27:17   
2               False  2020-07-25 12:27:14   
3               False  2020-07-25 12:27:10   
4               False  2020-07-25 12:27:08   
...               ...                  ...   
140096           True  2020-08-16 01:56:08   
140097          False  2020-08-16 01:56:00   
140098          False  2020-08-16 01:55:59   
140099          False  2020-08-16 01:55:58   
140100          False  2020-08-16 01:55:57   

                                                     text  \
0       If I smelled the scent of hand sanitizers toda...   
1       Hey @Yankees @YankeesPR and @MLB - wouldn't it...   
2       @diane3443 @wdunlap @realDonaldTrump Trump nev...   
3       @brookbanktv The one gift #COVID19 has give me...   
4       25 July : Media Bulletin on Novel #CoronaVirus...   
...                                                   ...   
140096  6) Meanwhile, the Institut national de santé p...   
140097  17912147095473612, 15/08/2020 12:43:02, Studen...   
140098  18115162954178167, 15/08/2020 13:16:55, Studen...   
140099  17899557754536668, 15/08/2020 13:16:57, Studen...   
140100  17851308170238866, 15/08/2020 13:17:08, Studen...   

                                 hashtags               source  is_retweet  
0                                     NaN   Twitter for iPhone       False  
1                                     NaN  Twitter for Android       False  
2                             ['COVID19']  Twitter for Android       False  
3                             ['COVID19']   Twitter for iPhone       False  
4       ['CoronaVirusUpdates', 'COVID19']  Twitter for Android       False  
...    

##### No duplicates in this dataset

In [10]:
covid_data.dtypes

user_name           object
user_location       object
user_description    object
user_created        object
user_followers       int64
user_friends         int64
user_favourites      int64
user_verified         bool
date                object
text                object
hashtags            object
source              object
is_retweet            bool
dtype: object

In [11]:
#Average number of followers user_followers
user_followers_avg = np.mean(covid_data['user_followers'])
user_followers_avg

108665.41141033969

In [12]:
#Average number of user_friends
user_friends_avg = np.mean(covid_data['user_friends'])
user_friends_avg

2104.862384993683

In [13]:
#Average number of user_favorites
user_favourites_avg = np.mean(covid_data['user_favourites'])
user_favourites_avg

14257.867802513902

In [14]:
#unique types of hashtags
print(covid_data['hashtags'].nunique())
print(covid_data['hashtags'].unique())

42175
[nan "['COVID19']" "['CoronaVirusUpdates', 'COVID19']" ...
 "['NEWS', 'NZ', 'COVID19nz', 'COVID19Aus', 'COVID19']"
 "['bbcnews', 'c4news', 'Newsnight', 'skynews', 'skypapers', 'covid19', 'coronavirus', 'covid19UK']"
 "['Chernobyl', 'COVID19']"]


In [34]:
#How many has the word 'news' in the hashtag
news_in_hashtag = []
for hashtag in covid_data['hashtags']:
    if type(hashtag) == float:
        news_in_hashtag.append(0)
    elif 'news' in hashtag or 'News' in hashtag:
        news_in_hashtag.append(1)
    else:
        news_in_hashtag.append(0)

print('number of hashtags containing "news": {}'.format(sum(news_in_hashtag)))

number of hashtags containing "news": 1204


In [35]:
#adding news_in_hashtag into the dataframe
covid_data['news_in_hashtag'] = news_in_hashtag

In [32]:
#How many has the word 'covid' in the hashtag
covid_in_hashtag = []
for hashtag in covid_data['hashtags']:
    if type(hashtag) == float:
        covid_in_hashtag.append(0)
    elif 'Covid' in hashtag or 'covid' in hashtag:
        covid_in_hashtag.append(1)
    else:
        covid_in_hashtag.append(0)

print('number of hashtags containing "covid": {}'.format(sum(covid_in_hashtag)))

number of hashtags containing "covid": 17086


In [33]:
#adding covid_in_hashtag into the dataframe
covid_data['covid_in_hashtag'] = covid_in_hashtag

In [17]:
covid_data['hashtags'].isnull().sum()

40162

In [18]:
#Number of unique users
print('number of unique user_names: ', covid_data['user_name'].nunique())
covid_data['user_name'].unique()

number of unique user_names:  76144


array(['ᏉᎥ☻լꂅϮ', 'Tom Basile 🇺🇸', 'Time4fisticuffs', ...,
       'The REALLY Old Guard', 'RANT-A-DAD', 'Pierre Alozie'],
      dtype=object)

In [19]:
#How many Verified Users
covid_data[covid_data['user_verified']==True].count()

user_name           18295
user_location       16788
user_description    18287
user_created        18295
user_followers      18295
user_friends        18295
user_favourites     18295
user_verified       18295
date                18295
text                18295
hashtags            13812
source              18295
is_retweet          18295
dtype: int64

In [20]:
#How many Unverified Users
covid_data[covid_data['user_verified']==False].count()

user_name           121806
user_location        94899
user_description    113847
user_created        121806
user_followers      121806
user_friends        121806
user_favourites     121806
user_verified       121806
date                121806
text                121806
hashtags             86127
source              121761
is_retweet          121806
dtype: int64

In [21]:
#Types of sources
print('number of unique sources: ', covid_data['source'].nunique())
# covid_data['source'].unique()

number of unique sources:  564


In [22]:
#Which sources are the tweets from? Iphone or Android?
tweet_source = []
for source in covid_data['source'][:6]:
    print(source)

Twitter for iPhone
Twitter for Android
Twitter for Android
Twitter for iPhone
Twitter for Android
Twitter Web App


In [24]:
#Setting up the spark dataframe for Querying
covid19_df = spark.read.csv('covid19_tweets.csv',
                           header = True,
                           sep = ',',
                           inferSchema = True)

covid19_df.createOrReplaceTempView('covid19_tweets')

In [25]:
covid19_df.printSchema()

root
 |-- user_name: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_created: string (nullable = true)
 |-- user_followers: string (nullable = true)
 |-- user_friends: string (nullable = true)
 |-- user_favourites: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- source: string (nullable = true)
 |-- is_retweet: string (nullable = true)



In [26]:
covid19_df.show(10)

+--------------------+--------------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+-------------------+----------+
|           user_name|       user_location|    user_description|       user_created|user_followers|user_friends|user_favourites|user_verified|               date|                text|            hashtags|             source|is_retweet|
+--------------------+--------------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+-------------------+----------+
|             ᏉᎥ☻լꂅϮ|          astroworld|wednesday addams ...|2017-05-26 05:46:42|           624|         950|          18775|        False|2020-07-25 12:27:21|If I smelled the ...|                null| Twitter for iPhone|     False|
|     Tom Basile 🇺🇸|        New York, NY|Husband, Father,

In [29]:
#Creates a text label about each bar in *rects*, displaying its height
def autolabel_percent(rects, orientation='vert'):
    #Prints percentage above bars for vertical bars
    if orientation=='vert':
        for rect in rects:
            height - rect.get_height()
            ax.annotate('{0:2f}%'.format(height),
                       xy=(rect.get_x()+rect.get_width()/2, height),
                       xytext=(0,3),
                       weight='bold',
                       textcoords='offset points',
                       ha='center', va='botton',
                       size=15)
    elif orientation=='hort':
        for rect in rects:
            width = rect.get_width()
            ax.annotate("{0:2f}%".format(width),
                       xy=(width, rect.get_y() + rect.get_height()/2),
                       xytext=(3,-6),
                       textcoords='offset points',
                       size=15)

In [30]:
#Print functions for baar graphs specifically
def print_bar(x, y, x_label, y_label, title='insert title', orientation='vert', color='blue',
             width=0.65, fig_size=(23,7), percentage=False, weight='bold', tick_size=20, title_size=30):
    
    #TODO: might have a problem with bars_for_annotation being called in scope
    fig, ax = plt.subplots(figsize=fig_size)
    if orientation=='vert':
        bars_for_annotation = ax.bar(x, y, color=color, align='center', width=width)
    elif orientation=='hort':
        bars_for_annotation = ax.barh(x, y, color=color, align='center')
        
    plt.xticks(size=13, rotation=90)
    plt.yticks(size=15)
    plt.xlabel(x_label, size=tick_size, color=color)
    plt.ylabel(y_label, size=tick_size, color=color)
    plt.title(title, fontsize=title_size, color=color)
    
    autolabel_percentage(bars_for_annotation, orientation)
    
    plt.show()

In [ ]:
#Converting pandas df to spark sql